In [1]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import DictCursor
import pandas as pd

In [2]:
# Function that connects to database
load_dotenv()
def db_connect():
    db_name = os.environ['db_name']
    db_user = os.environ['db_user']
    db_host = os.environ['db_host']
    db_credentials = os.environ['db_creds']
  
    conn_string = "dbname='" + str(db_name) + "' user='" + str(db_user) + "' host='" + str(db_host) + "' password='" + str(db_credentials) + "'"

    try:
        conn = psycopg2.connect(str(conn_string))
        conn.autocommit = True
    except:
        print("Unable to connect to the database")

    cur = conn.cursor(cursor_factory=DictCursor)
    return cur

cur = db_connect()

In [3]:
# Load weather data to DataFrame
query = "SELECT * FROM weather.weather_clean"
cur.execute(query)
results = cur.fetchall()

weather_df = pd.DataFrame(results, columns=['date','tmin','tmax','tavg','prcp','snow','snwd'])
weather_df.head(10)

,date,tmin,tmax,tavg,prcp,snow,snwd
0,1990-01-02,38.485437,52.512438,45.498937,0.114851,0.017143,5.950000
1,1990-01-03,38.889423,52.830000,45.859712,0.157523,0.028571,6.868085
2,1990-01-04,37.970732,52.688119,45.329425,0.153816,0.040000,7.067391
3,1990-01-05,37.485714,52.746411,45.116063,0.075048,0.034286,6.489362
4,1990-01-06,37.658654,53.730233,45.694443,0.116303,0.062857,5.844186
5,1990-01-08,37.886792,51.191919,44.539356,0.088450,0.194286,6.108696
6,1990-01-09,36.970149,53.210784,45.090467,0.104313,0.040000,6.900000
7,1990-01-10,38.344186,52.229268,45.286727,0.146854,0.262857,6.119565
8,1990-01-11,38.732057,52.473430,45.602744,0.117689,0.037143,6.995652
9,1990-01-12,39.288372,53.346341,46.317357,0.193529,0.297143,7.502273


In [4]:
# Load disaster data to DataFrame
query = "SELECT * FROM weather.disasters_clean"
cur.execute(query)
results = cur.fetchall()

disaster_df = pd.DataFrame(results, columns=['disaster_no','year','country_code','region','disaster_type','deaths','damages'])
disaster_df.head(10)

,disaster_no,year,country_code,region,disaster_type,deaths,damages
0,1900-0009-TUR,1900,TR,Western Asia,Earthquake,140.0,NaN
1,1900-0008-JPN,1900,JP,Eastern Asia,Volcanic activity,30.0,NaN
2,1900-0003-USA,1900,US,Northern America,Storm,6000.0,30000.0
3,1900-0007-JAM,1900,JM,Caribbean,Epidemic,30.0,NaN
4,1900-0006-JAM,1900,JM,Caribbean,Flood,300.0,NaN
5,1901-0001-UGA,1901,UG,Eastern Africa,Epidemic,200000.0,NaN
6,1901-0004-JPN,1901,JP,Eastern Asia,Earthquake,18.0,NaN
7,1902-0014-CHN,1902,CN,Eastern Asia,Earthquake,2500.0,NaN
8,1902-0013-AZE,1902,AZ,Western Asia,Earthquake,86.0,NaN
9,1902-0010-GTM,1902,GT,Central America,Volcanic activity,6000.0,NaN
